In [1]:
### PREREQUISTIES
import pandas as pd
from bs4 import BeautifulSoup
import random
import requests
import getpass
import time
from github import Github
import sddk

In [9]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


# Get paths and filenames for a GitHub repo...

We will extract our text files directly from github, without cloning the repositories containing the texts.

In [3]:
### make a github authorization
g = Github("kasev", getpass.getpass("github password: "))

We first go to `canonical-greekLit` repo from PerseusDO.

In [4]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("PerseusDL").get_repo("canonical-greekLit" )
paths_perseus = []
contents = repo.get_contents("data")
while contents:
  file_content = contents.pop(0)
  if file_content.type == "dir":
    contents.extend(repo.get_contents(file_content.path))
  else:
    try:
      path_and_name = file_content.path
      if ".xml" in str(path_and_name):
        if "perseus-grc" in str(path_and_name):
          #print(path_and_name)
          paths_perseus.append(str(path_and_name))
    except: 
      pass

CPU times: user 3.98 s, sys: 466 ms, total: 4.44 s
Wall time: 7min 26s


In [5]:
len(paths_perseus)

810

In [8]:
paths_perseus[:5]

['data/tlg0001/tlg001/tlg0001.tlg001.perseus-grc2.xml',
 'data/tlg0003/tlg001/tlg0003.tlg001.perseus-grc2.xml',
 'data/tlg0004/tlg001/tlg0004.tlg001.perseus-grc2.xml',
 'data/tlg0005/tlg001/tlg0005.tlg001.perseus-grc2.xml',
 'data/tlg0005/tlg002/tlg0005.tlg002.perseus-grc2.xml']

In [10]:
s.write_file("SDAM_data/AGT/perseus_paths_20240104.json", paths_perseus)

Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/perseus_paths_20240104.json"


In [11]:
%%time
repo = g.get_user("OpenGreekAndLatin").get_repo("First1KGreek" )
paths_ogl = []
contents = repo.get_contents("data")
while contents:
  file_content = contents.pop(0)
  if file_content.type == "dir":
    contents.extend(repo.get_contents(file_content.path))
  else:
    try:
      path_and_name = file_content.path
      if ".xml" in str(path_and_name):
        if "__cts__" not in str(path_and_name):    
          paths_ogl.append(str(path_and_name))
    except: 
      pass

CPU times: user 6.52 s, sys: 752 ms, total: 7.27 s
Wall time: 10min 53s


In [13]:
paths_ogl[:5]

['data/ggm0001/ggm001/ggm0001.ggm001.1st1K-grc1.xml',
 'data/heb0001/heb010/heb0001.heb010.1st1K-eng1.xml',
 'data/ogl0001/ogl001/ogl0001.ogl001.1st1K-grc1.xml',
 'data/stoa0033a/tlg028/stoa0033a.tlg028.1st1K-grc1.xml',
 'data/stoa0033a/tlg043/stoa0033a.tlg043.1st1K-grc1.xml']

In [15]:
s.write_file("SDAM_data/AGT/ogl_paths_20240104.json", paths_ogl)

Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/ogl_paths_20240104.json"


## Testing with individual texts from xml files

In [19]:
response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0086/tlg010/tlg0086.tlg010.perseus-grc1.xml')
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()[:3000])

<?xml version="1.0" encoding="UTF-8"?>
<tei.2>
 <teiheader>
  <filedesc>
   <titlestmt>
    <title>
     Nicomachean Ethics
    </title>
    <title type="sub">
     Machine readable text
    </title>
    <author>
     Aristotle
    </author>
    <editor>
     J. Bywater
    </editor>
    <sponsor>
     Perseus Project, Tufts University
    </sponsor>
    <principal>
     Gregory Crane
    </principal>
    <respstmt>
     <resp>
      Prepared under the supervision of
     </resp>
     <name>
      Lisa Cerrato
     </name>
     <name>
      William Merrill
     </name>
     <name>
      Elli Mylonas
     </name>
     <name>
      David Smith
     </name>
    </respstmt>
    <funder n="org:AnnCPB">
     The Annenberg CPB/Project
    </funder>
   </titlestmt>
   <publicationstmt>
    <publisher>
     Trustees of Tufts University
    </publisher>
    <pubplace>
     Medford, MA
    </pubplace>
    <authority>
     Perseus Project
    </authority>
   </publicationstmt>
   <sourcedesc>
    

In [21]:
soup.titlestmt.title.get_text()

'Nicomachean Ethics'

In [23]:
all_p = soup.body.find_all("p")
title = soup.titlestmt.title.get_text()
raw_text = []
for p in all_p:
  raw_text.append(p.get_text())
raw_text_str = " ".join(raw_text)
raw_text_str[:3000]

'πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς τε καὶ προαίρεσις, ἀγαθοῦ τινὸς ἐφίεσθαι δοκεῖ· διὸ καλῶς ἀπεφήναντο τἀγαθόν, οὗ πάντʼ ἐφίεται. διαφορὰ δέ τις φαίνεται τῶν τελῶν· τὰ μὲν γάρ εἰσιν ἐνέργειαι, τὰ δὲ παρʼ αὐτὰς ἔργα τινά. ὧν δʼ εἰσὶ τέλη τινὰ παρὰ τὰς πράξεις, ἐν τούτοις βελτίω πέφυκε τῶν ἐνεργειῶν τὰ ἔργα. πολλῶν δὲ πράξεων οὐσῶν καὶ τεχνῶν καὶ ἐπιστημῶν πολλὰ γίνεται καὶ τὰ τέλη· ἰατρικῆς μὲν γὰρ ὑγίεια, ναυπηγικῆς δὲ πλοῖον, στρατηγικῆς δὲ νίκη, οἰκονομικῆς δὲ πλοῦτος. ὅσαι δʼ εἰσὶ τῶν τοιούτων ὑπὸ μίαν τινὰ δύναμιν, καθάπερ ὑπὸ τὴν ἱππικὴν χαλινοποιικὴ καὶ ὅσαι ἄλλαι τῶν ἱππικῶν ὀργάνων εἰσίν, αὕτη δὲ καὶ πᾶσα πολεμικὴ πρᾶξις ὑπὸ τὴν στρατηγικήν, κατὰ τὸν αὐτὸν δὴ τρόπον ἄλλαι ὑφʼ ἑτέρας· ἐν ἁπάσαις δὲ τὰ τῶν ἀρχιτεκτονικῶν τέλη πάντων ἐστὶν αἱρετώτερα τῶν ὑπʼ αὐτά· τούτων γὰρ χάριν κἀκεῖνα διώκεται. διαφέρει δʼ οὐδὲν τὰς ἐνεργείας αὐτὰς εἶναι τὰ τέλη τῶν πράξεων ἢ παρὰ ταύτας ἄλλο τι, καθάπερ ἐπὶ τῶν λεχθεισῶν ἐπιστημῶν. εἰ δή τι τέλος ἐστὶ τῶν πρακτῶν ὃ διʼ αὑτὸ βουλό

## Testing with a sample

In [24]:
### start with the paths
#paths_ogl = s.read_file("SDAM_data/AGT/ogl_paths_20240104.json", "list")
#paths_perseus = s.read_file("SDAM_data/AGT/perseus_paths_20240104.json", "list")

In [37]:
%%time


## testing
data = []
for filepath in random.choices(paths_ogl, k=10):
  response = requests.get('https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/' + filepath)
  soup = BeautifulSoup(response.text, "html.parser")
  try:
    try:
        raw_text = []
        all_p = soup.body.find_all("p")
        for p in all_p:
            raw_text.append(p.get_text())
        string = " ".join(raw_text)
    except:
        string = " ".join(soup.body.text.split())
  except:
    string = "" 
  wordcount = len(string.split())
  try:
    author = soup.author.text
  except:
    author = None
  try:
    title = soup.titlestmt.title.get_text()
  except:
    title = None
  data.append([filepath.rpartition("/")[2], author, title, string, wordcount])
print(len(data))

10
CPU times: user 622 ms, sys: 36.8 ms, total: 659 ms
Wall time: 3.96 s


In [38]:
pd.DataFrame(data)

,0,1,2,3,4
0,tlg1765.tlg003.1st1K-grc1.xml,Vitae Aesopi,Vita Aesopi Pl vel Accursiana (sub auctore Max...,Πραγμάτων φύσιν τῶν ἐν ἀνθρώποις ἠκρίβωσαν\nμὲ...,27370
1,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867
2,tlg0592.tlg005.1st1K-grc1.xml,Hermogenes,Περὶ μεθόδου δεινότητος,1 Πᾶν μέρος λόγου εὕρηται μὲν ἐπὶ μηνύσει πράγ...,11248
3,tlg4026.tlg001.1st1K-grc1.xml,Anonymi in Aristotelis Artem Rhetoricam,In Aristotelis artem rhetoricam commentarium,[1356b 34] καὶ ἡ διαλεκτικὴ οὐ σκοπεῖ τὸ δοκοῦ...,100201
4,tlg0643.tlg001.1st1K-grc2.xml,Hermann Diels,Sigla to Anonymus Londinensis Iatrica,"ABBREV = αἴτιος, αἰτία cet. ABBREV = ἀλλά(ἄλλη...",110
5,tlg1595.tlg370.1st1K-grc1.xml,Philodemus,Volumina Rhetorica,"ἢ πρὸς τὰς κατὰ μέρος λόγων διακοπάς, ἐπιμνησθ...",9828
6,tlg0081.tlg006.1st1K-grc1.xml,Dionysius of Halicarnassus,De Demosthenis dictione,\nδικανικοῖς μὲν οὖν οὐ περιέτυχον αὐτοῦ λόγοι...,35383
7,tlg0527.tlg048.1st1K-eng3.xml,Septuaginta,Index II to Isaias,"(References either to pages, or to chapters an...",3849
8,tlg2042.tlg052.1st1K-grc1.xml,Origenes,Selecta in Numeros,Οὐδὲν πλὴν εἰς τὸ μάννα οἱ ὀφθαλμοὶ ἡμῶ. Μᾶν ὠ...,1804
9,tlg2018.tlg007.1st1K-grc1.xml,Eusebius of Caesarea,των κατά Μαρκέλλου,Λόγος Α′. ⟨αʹ πόθεν ὁρμώμενος ὁ Μάρκελλος του...,18376


In [36]:
data[1][3][:1000]

'Κατ᾽ ἐκεῖνον τὸν καιρὸν ἦμεν πάντες οἱ ἀπόστολοι ἐν Ἰεροσολύμοις, Σίμων ὁ λεγόμενος Πέτρος καὶ Ἀνδρέας ὁ ἀδελφὸς αὐτοῦ, Ἰάκωβος ὁ τοῦ Ζεβεδαίου καὶ Ἰωάννης ὁ ἀδελφὸς αὐτοῦ, Φίλιππος καὶ Βαρθολομαῖος, Θωμᾶς καὶ Ματθαῖος ὁ τελώνης, Ἰάκωβος Ἁλφαίου καὶ Σίμων ὁ Καναναῖος,  καὶ Ἰούδας Ἰακώβου, καὶ διείλαμεν τὰ κλίματα τῆς οἰκουμένης, ὅπως εἷς ἕκαστος ἡμῶν ἐν τῷ κλίματι τῷ λαχόντι αὐτῷ καὶ εἰς τὸ ἔθνος ἐν ᾧ ὁ κύριος αὐτὸν ἀπέστειλεν πο ρευθῇ. κατὰ κλῆρον οὖν ἔλαχεν ἡ Ἰνδία Ἰούδᾳ Θωμᾷ τῷ καὶ Διδύμῳ· οὐκ ἐβούλετο δὲ ἀπελθεῖν, λέγων μὴ δύνασθαι μήτε χωρεῖν διὰ τὴν ἀσθένειαν τῆς σαρκός, καὶ ὅτι Ἄνθρωπος ὢν Ἑβραῖος πῶς δύναμαι πορευθῆναι ἐν τοῖς Ἰνδοῖς κηρύξαι τὴν ἀλήθειαν; Καὶ ταῦτα αὐτοῦ διαλογιζομένου καὶ λέγοντος ὤφθη αὐτῷ ὁ σωτὴρ διὰ τῆς νυκτός, καὶ λέγει αὐτῷ· Μὴ φοβοῦ Θωμᾶ, ἄπελθε εἰς τὴν Ἰνδίαν καὶ κήρυξον ἐκεῖ τὸν λόγον· ἡ γὰρ χάρις μού ἐστιν μετὰ σοῦ. Ὃ δὲ οὐκ ἐπείθετο, λέγων· Ὅπου βούλει με ἀποστεῖλαι ἀπόστειλον ἀλλαχοῦ· εἰς Ἰνδοὺς γὰρ οὐκ ἀπέρχομαι.  Καὶ ταῦτα αὐτοῦ λέγοντος καὶ ἐ

## Extraction of the whole dataset

In [39]:
%%time
## actual extraction

data = []
for filepath in paths_perseus:
    try:
        response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/' + filepath)  
        soup = BeautifulSoup(response.text, "html.parser")
    except:
        time.sleep(1)
        response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/' + filepath)  
        soup = BeautifulSoup(response.text, "html.parser")
    try:
        # try paragraphs
        raw_text = []
        # extract p string
        all_p = soup.body.find_all("p")
        for p in all_p:
            raw_text.append(p.get_text())
        p_string = " ".join(raw_text)
        # extract l string
        all_l = soup.body.find_all("l")
        raw_text = []
        all_l = soup.body.find_all("l")
        for line in all_l:
            raw_text.append(line.get_text())
        l_string = " ".join(raw_text)
        body_string = " ".join(soup.body.text.split())
        #test: 
        #if (4 > (10 /2)) | (4 > (10 /2)):
        #    print("true")
        if (len(l_string) > (len(body_string)/2)) | (len(p_string) > (len(body_string)/2)):
            if len(l_string) > len(p_string):
                string = l_string
            else:
                string = p_string
        else:
            string = body_string
    except: 
        string = ""
    wordcount = len(string.split())
    try:
        author = soup.author.text
    except:
        author = None
    try:
        title = soup.titlestmt.title.get_text()
    except:
        title = None
    data.append([filepath.rpartition("/")[2], author, title, string, wordcount])
print(len(data))

810
CPU times: user 59.8 s, sys: 1.84 s, total: 1min 1s
Wall time: 5min 2s


In [41]:
data_perseus = pd.DataFrame(data, columns=["filename", "author", "title", "string", "wordcount"])
data_perseus.head(30)

,filename,author,title,string,wordcount
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",38822
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,Ἱστορίαι,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν\...,150363
2,tlg0004.tlg001.perseus-grc2.xml,Diogenes Laertius,Βίοι καὶ γνῶμαι τῶν ἐν φιλοσοφίᾳ εὐδοκιμησάντων,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,110671
3,tlg0005.tlg001.perseus-grc2.xml,Theocritus,Εἰδύλλια,"Ἁδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα, ...",19435
4,tlg0005.tlg002.perseus-grc2.xml,Theocritus,Ἐπιγράμματα,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,1750
5,tlg0006.tlg001.perseus-grc2.xml,Euripides,Κύκλωψ,"Ὦ Βρόμιε, διὰ σὲ μυρίους ἔχω πόνους νῦν χὥτ’...",4141
6,tlg0006.tlg002.perseus-grc2.xml,Euripides,Ἄλκηστις,"Ὦ δώματ’ Ἀδμήτειʼ, ἐν οἷς ἔτλην ἐγὼ θῆσσαν τ...",6602
7,tlg0006.tlg003.perseus-grc2.xml,Euripides,Μήδεια,Εἴθ’ ὤφελ’ Ἀργοῦς μὴ διαπτάσθαι σκάφος Κόλχων...,8026
8,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,Πάλαι ποτʼ ἐστὶ τοῦτʼ ἐμοὶ δεδογμένον· ὁ μὲν...,6272
9,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,"Πολλὴ μὲν ἐν βροτοῖσι κοὐκ ἀνώνυμος, θεὰ κέκλ...",8257


In [42]:
data_perseus[data_perseus["wordcount"]<200]

,filename,author,title,string,wordcount
221,tlg0013.tlg001.perseus-grc2.xml,Anonymous,Hymn 1 To Dionysus,"\n\nοἳ μὲν γὰρ Δρακάνῳ σʼ, οἳ δʼ Ἰκάρῳ ἠνεμοέσ...",144
226,tlg0013.tlg006.perseus-grc2.xml,Anonymous,Hymn 6 To Aphrodite,"αἰδοίην, χρυσοστέφανον, καλὴν Ἀφροδίτην ᾁσομα...",131
228,tlg0013.tlg008.perseus-grc2.xml,Anonymous,Hymn 8 To Ares,"Ἆρες ὑπερμενέτα, βρισάρματε, χρυσεοπήληξ, ὀβρ...",100
229,tlg0013.tlg009.perseus-grc2.xml,Anonymous,Hymn 9 To Artemis,"Ἄρτεμιν ὕμνει, Μοῦσα, κασιγνήτην Ἑκάτοιο. παρ...",57
230,tlg0013.tlg010.perseus-grc2.xml,Anonymous,Hymn 10 To Aphrodite,"κυπρογενῆ Κυθέρειαν ἀείσομαι, ἥτε βροτοῖσι με...",39
231,tlg0013.tlg011.perseus-grc2.xml,Anonymous,Hymn 11 To Athena,"Παλλάδʼ Ἀθηναίην ἐρυσίπτολιν ἄρχομʼ ἀείδειν, ...",35
232,tlg0013.tlg012.perseus-grc2.xml,Anonymous,Hymn 12 To Hera,"Ἥρην ἀείδω χρυσόθρονον, ἣν τέκε Ῥείη, ἀθανάτω...",28
233,tlg0013.tlg013.perseus-grc2.xml,Anonymous,Hymn 13 To Demeter,"Δημήτηρ’ ἠύκομον, σεμνὴν θεάν, ἄρχομ’ ἀείδειν,...",20
234,tlg0013.tlg014.perseus-grc2.xml,Anonymous,Hymn 14 to the Mother of the Gods,μητέρα μοι πάντων τε θεῶν πάντων τʼ ἀνθρώπων ...,46
235,tlg0013.tlg015.perseus-grc2.xml,Anonymous,Hymn 15 To Heracles the Lion-Hearted,"Ἡρακλέα, Διὸς υἱόν, ἀείσομαι, ὃν μέγʼ ἄριστον ...",61


In [43]:
data_perseus[data_perseus["string"] == ""]

,filename,author,title,string,wordcount


In [44]:
s.write_file("SDAM_data/AGT/data_perseus_20240104.json", data_perseus)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/data_perseus_20240104.json"


In [45]:
paths_ogl[:10]

['data/ggm0001/ggm001/ggm0001.ggm001.1st1K-grc1.xml',
 'data/heb0001/heb010/heb0001.heb010.1st1K-eng1.xml',
 'data/ogl0001/ogl001/ogl0001.ogl001.1st1K-grc1.xml',
 'data/stoa0033a/tlg028/stoa0033a.tlg028.1st1K-grc1.xml',
 'data/stoa0033a/tlg043/stoa0033a.tlg043.1st1K-grc1.xml',
 'data/stoa0121/stoa001/stoa0121.stoa001.opp-grc1.xml',
 'data/stoa0146d/stoa001/stoa0146d.stoa001.opp-grc1.xml',
 'data/stoa0146d/stoa001/stoa0146d.stoa001.opp-lat1.xml',
 'data/tlg0005/tlg003/tlg0005.tlg003.1st1K-grc1.xml',
 'data/tlg0006/tlg020/tlg0006.tlg020.1st1K-grc1.xml']

In [46]:
response = requests.get('https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/data/tlg0005/tlg003/tlg0005.tlg003.1st1K-grc1.xml')
soup = BeautifulSoup(response.text, "html.parser")

In [47]:
%%time
# OGL

data = []
for filepath in paths_ogl:
    response = requests.get('https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/' + filepath)
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        # try paragraphs
        raw_text = []
        # extract p string
        all_p = soup.body.find_all("p")
        for p in all_p:
            raw_text.append(p.get_text())
        p_string = " ".join(raw_text)
        # extract l string
        raw_text = []
        all_l = soup.body.find_all("l")
        for line in all_l:
            raw_text.append(line.get_text())
        l_string = " ".join(raw_text)
        body_string = " ".join(soup.body.text.split())
        #test: 
        #if (4 > (10 /2)) | (4 > (10 /2)):
        #    print("true")
        if (len(l_string) > (len(body_string)/2)) | (len(p_string) > (len(body_string)/2)):
            if len(l_string) > len(p_string):
                string = l_string
            else:
                string = p_string
        else:
            string = body_string
    except: 
        string = ""
    wordcount = len(string.split())
    try:
        author = soup.author.text
    except:
        author = None
    try:
        try:
            title = soup.titlestmt.title.get_text()
        except:
            title = None
            title = ""
        #for t in soup.find_all("title"):
        #    title = title + "; " + t.get_text()
        #title = title[2:]
    except:
        title = None
    data.append([filepath.rpartition("/")[2], author, title, string, wordcount])
print(len(data))

1201
CPU times: user 1min 35s, sys: 2.9 s, total: 1min 38s
Wall time: 7min 59s


In [48]:
data_ogl = pd.DataFrame(data, columns=["filename", "author", "title", "string", "wordcount"])
data_ogl.head(30)

,filename,author,title,string,wordcount
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias,FROM HEBREW I. 1 The vision of Isaiah the son ...,40168
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459
5,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867
6,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166
7,stoa0146d.stoa001.opp-lat1.xml,Hegemonius,Acta Archelai,Thesaurus verus sive disputatio habita in Carc...,42159
8,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77
9,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708


In [50]:
s.write_file("SDAM_data/AGT/data_ogl_20240104.json", data_ogl)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/data_ogl_20240104.json"


# Read and Merge the Data

In [51]:
### READ THE DATA FROM SCIENCEDATA.DK
### it takes quite long, for some reason...

In [52]:
#data_perseus = s.read_file("SDAM_data/AGT/data_perseus_20201026.json", "df")
#data_ogl= s.read_file("SDAM_data/AGT/data_ogl_20201026.json", "df")

First, we have to explore whether there is an overlap in file names between the two corpora

In [53]:
### produce a list of filenames in the perseus dataset
filenames_perseus = data_perseus.filename.unique().tolist()
print(filenames_perseus[:10])

['tlg0001.tlg001.perseus-grc2.xml', 'tlg0003.tlg001.perseus-grc2.xml', 'tlg0004.tlg001.perseus-grc2.xml', 'tlg0005.tlg001.perseus-grc2.xml', 'tlg0005.tlg002.perseus-grc2.xml', 'tlg0006.tlg001.perseus-grc2.xml', 'tlg0006.tlg002.perseus-grc2.xml', 'tlg0006.tlg003.perseus-grc2.xml', 'tlg0006.tlg004.perseus-grc2.xml', 'tlg0006.tlg005.perseus-grc2.xml']


In [54]:
works_perseus = [filename.rpartition(".")[0].rpartition(".")[0] for filename in filenames_perseus]
print(works_perseus[:10])

['tlg0001.tlg001', 'tlg0003.tlg001', 'tlg0004.tlg001', 'tlg0005.tlg001', 'tlg0005.tlg002', 'tlg0006.tlg001', 'tlg0006.tlg002', 'tlg0006.tlg003', 'tlg0006.tlg004', 'tlg0006.tlg005']


In [55]:
def check_filename(row_and_column):
  perseus = False
  for work in works_perseus:
    if row_and_column.startswith(work):
      perseus = True
      continue
  return perseus

In [56]:
len(data_ogl)

1201

In [57]:
data_ogl["in_perseus"] = data_ogl["filename"].apply(check_filename)

In [58]:
data_ogl["in_perseus"].sum()

44

In [60]:
data_ogl[data_ogl["in_perseus"]==True]

,filename,author,title,string,wordcount,in_perseus
12,tlg0011.tlg003.1st1K-grc1.xml,Sophocles,Ajax,"Ἀεὶ μὲν, ὦ παῖ Λαρτίου, δἐδορκά σε πεῖράν τι...",7982,True
79,tlg0031.tlg002.1st1K-cop1.xml,,Mark Chapter 1,ⲧ ⲁⲣⲭⲏ ⲙ ⲡ ⲉⲩⲁⲅⲅⲉⲗⲓⲟⲛ ⲛ ⲓⲏⲥⲟⲩⲥ ⲡⲉ ⲭⲣⲓⲥⲧⲟⲥ [ ⲡ ...,1231,True
80,tlg0031.tlg002.INTF-grc1.xml,None,01,αρχη του ευαγγελι ου ιυ χυ υυ θυ καθως γε γραπ...,831,True
100,tlg0057.tlg010.1st1K-grc1.xml,Galen,De naturalibus facultatibus,Ἐπειδὴ τὸ μὲν αἰσθάνεσθαί τε καὶ κινεῖσθαι κατ...,31877,True
213,tlg0081.tlg006.1st1K-grc1.xml,Dionysius of Halicarnassus,De Demosthenis dictione,\nδικανικοῖς μὲν οὖν οὐ περιέτυχον αὐτοῦ λόγοι...,35383,True
214,tlg0081.tlg007.1st1K-grc1.xml,Dionysius of Halicarnassus,Libri secundi de antiquis oratoribus reliquiae,\nDE DEMOSTHENE\n I 1 TZETZAE scholion in Cr...,261,True
224,tlg0085.tlg001.1st1K-grc1.xml,Aeschylus,Ἱκέτιδες,Ζεὺς μὲν ἀφίκτωρ ἐπίδοι προφρόνως στόλον ἡμέτε...,7627,True
225,tlg0085.tlg002.1st1K-grc1.xml,Aeschylus,Πέρσαι,Τάδε μὲν Περσῶν τῶν οἰχομένων Ἑλλάδ᾽ ἐς αἶαν π...,6877,True
226,tlg0085.tlg003.1st1K-grc1.xml,Aeschylus,Προμηθεὺς Δεσμώτης,"Χθονὸς μὲν ἐς τηλουρὸν ἥκομεν πέδον, Σκύθην ἐς...",7207,True
227,tlg0085.tlg004.1st1K-grc1.xml,Aeschylus,Ἑπτὰ ἐπὶ Θήβας,"Κάδμου πολῖται, χρὴ λέγειν τὰ καίρια ὅστις φυλ...",6844,True


In [66]:
data_ogl["in_perseus"] = data_ogl["filename"].apply(check_filename)
### to filter for values not in perseus
data_ogl = data_ogl[data_ogl["in_perseus"]==False]

In [67]:
data_ogl.drop(columns=["in_perseus"], inplace=True)
len(data_ogl)

1157

In [72]:
data_perseus["source"] = "perseus"
data_ogl["source"] = "1Kgr"

In [73]:
### finally, merge the two datasets (perseus + ogl)
data_merged_raw = pd.concat([data_ogl, data_perseus])

In [74]:
### how many files we
len(data_merged_raw)

1967

In [75]:
data_merged_raw.reset_index(inplace=True)
data_merged_raw.drop(columns=["index"], inplace=True)

In [76]:
data_merged_raw.head(20)

,filename,author,title,string,wordcount,source
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias,FROM HEBREW I. 1 The vision of Isaiah the son ...,40168,1Kgr
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr
5,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr
6,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166,1Kgr
7,stoa0146d.stoa001.opp-lat1.xml,Hegemonius,Acta Archelai,Thesaurus verus sive disputatio habita in Carc...,42159,1Kgr
8,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr
9,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr


In [77]:
data_merged_raw["wordcount"].sum()

40863400

In [78]:
s.write_file("SDAM_data/AGT/AGT_raw_20240104.json", data_merged_raw)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/AGT_raw_20240104.json"
